In [2]:
# fix notebook
import os, django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# Роутеры

### Определяем стандартные маршруты для viewset
[Документация](https://www.django-rest-framework.org/api-guide/routers/)

In [30]:
from rest_framework import generics, viewsets, mixins
from rest_framework.decorators import action
from paper.models import Paper
from paper.serializers import PaperSerializer
from rest_framework.viewsets import GenericViewSet

In [28]:
class PaperViewSet(viewsets.ReadOnlyModelViewSet):
    queryset = Paper.objects.all()
    serializer_class = PaperSerializer

In [32]:
from rest_framework import routers

router = routers.SimpleRouter() # создаем объект роутера

router.register(
    r'paper', # префикс для набора маршрутов
    PaperViewSet # класс viewset
)
print(router.urls)

[<URLPattern '^paper/$' [name='paper-list']>, <URLPattern '^paper/test_action/$' [name='paper-test-action']>, <URLPattern '^paper/(?P<pk>[^/.]+)/$' [name='paper-detail']>]


In [9]:
# прописываем в url.py набор маршрутов

from django.urls import path, include

urlpatterns = [
    path('api/v1', include(router.urls)) # включаем все маршруты которые у нас были в urls .../api/v1/paper/
]

In [23]:
# default router
from pprint import pprint

# <URLPattern '^paper/$' [name='paper-list']>,
# paper-list сгенерировано благодаря имени модели paper, чтобы поменять это имя нужно указать basename=''
# при этом, если queryset отсутствует, то параметр basename обязателен

router = routers.DefaultRouter()

router.register(
    r'paper', # префикс для набора маршрутов
    PaperViewSet # класс viewset
#     basename='paper'
)

pprint(router.urls)

# если вместо DefaultRouter использовать SimpleRouter, то маршрут /api/v1/ будет недоступен

[<URLPattern '^paper/$' [name='paper-list']>,
 <URLPattern '^paper\.(?P<format>[a-z0-9]+)/?$' [name='paper-list']>,
 <URLPattern '^paper/(?P<pk>[^/.]+)/$' [name='paper-detail']>,
 <URLPattern '^paper/(?P<pk>[^/.]+)\.(?P<format>[a-z0-9]+)/?$' [name='paper-detail']>,
 <URLPattern '^$' [name='api-root']>,
 <URLPattern '^\.(?P<format>[a-z0-9]+)/?$' [name='api-root']>]


In [14]:
pprint(router.urls[0:2]) # отвечает за список статей /api/v1/paper/ и возможностью добавлять новые статьи 
# если такой функицонал предусмотрен

[<URLPattern '^paper/$' [name='paper-list']>,
 <URLPattern '^paper\.(?P<format>[a-z0-9]+)/?$' [name='paper-list']>]


In [16]:
pprint(router.urls[2:4]) # эта группа маршрутов отвечает за удаление статьи и за ее изменение /api/v1/paper/pk

[<URLPattern '^paper/(?P<pk>[^/.]+)/$' [name='paper-detail']>,
 <URLPattern '^paper/(?P<pk>[^/.]+)\.(?P<format>[a-z0-9]+)/?$' [name='paper-detail']>]


In [22]:
pprint(router.urls[4::]) # существует только в default router возращает список маршрутов которые существуют в роутере
# в SimpleRouter такого нет

[<URLPattern '^$' [name='api-root']>,
 <URLPattern '^\.(?P<format>[a-z0-9]+)/?$' [name='api-root']>]


In [24]:
# <URLPattern '^paper/$' [name='paper-list']>
# также можно везде заметить префикс paper-list он берется по имени модели 'from paper.models import Paper'
# мы можем поменять только префикс маршрута

In [37]:
class PaperViewSet(mixins.CreateModelMixin,
                   mixins.RetrieveModelMixin,
                   mixins.UpdateModelMixin,
                   mixins.DestroyModelMixin,
                   mixins.ListModelMixin,
                   GenericViewSet):
    queryset = Paper.objects.all()
    serializer_class = PaperSerializer

    
    # если стандартных маршрутов недостаточно, то можно использовать декоратор action

    # маршрут будет выглядеть таким образом ..../api/v1/paper/test_action/
    @action(methods=['get'], detail=False)
    def categories(self, request): # вывод всех категорий
        cats = Categories.objects.all()
        return Response({"cats": [c.name for c in cats]})
    
    @action(methods=['get'], detail=True) # вывод определенной категории по идентификатору 
    # если detail=True значит выводится 1 запись
    def category(self, request, pk=None):
        cats = Paper.objects.get(pk=pk)
        return Response({'cat': cats.name})

In [38]:
# Если по маршруту ...api/v1/paper/ мы хотим возвращать какую-то часть данных или хотим сделать сложную группировку то можно 
# переопределить метод get_queryset

class PaperViewSet(mixins.CreateModelMixin,
                   mixins.RetrieveModelMixin,
                   mixins.UpdateModelMixin,
                   mixins.DestroyModelMixin,
                   mixins.ListModelMixin,
                   GenericViewSet):
    # queryset = Paper.objects.all() теперь этот атрибут не нужен, поскольку у нас есть get_queryset
    serializer_class = PaperSerializer

    
    def get_queryset(self):
        pk = self.kwargs.get('pk')
        if not pk:
            return Paper.objects.all() # возвращаем только первые 2 записи из таблицы
        return Paper.objects.filter(pk=pk)

    
    # если стандартных маршрутов недостаточно, то можно использовать декоратор action
    # маршрут будет выглядеть таким образом ..../api/v1/paper/test_action/
    @action(methods=['get'], detail=False)
    def categories(self, request): # вывод всех категорий
        cats = Categories.objects.all()
        return Response({"cats": [c.name for c in cats]})
    
    @action(methods=['get'], detail=True) # вывод определенной категории по идентификатору 
    # если detail=True значит выводится 1 запись
    def category(self, request, pk=None):
        cats = Paper.objects.get(pk=pk)
        return Response({'cat': cats.name})

Роутер в Django - это набор правил, которые определяют, 
как URL-адреса должны быть разбиты на компоненты и как они должны быть переданы в функции представления.
Роутеры используются для обработки запросов к веб-приложению Django.

1. Route - объект, который описывает обычный маршрут (не action) с методами list и retrieve.
2. url - шаблон URL для маршрута, в котором можно использовать переменные, обозначенные фигурными скобками.
3. mapping - словарь, который связывает метод HTTP с методом APIView.
4. name - имя маршрута, которое можно использовать для генерации URL.
5. detail - определяет, является ли маршрут детальным (с переменной в URL) или нет.
6. initkwargs - дополнительные аргументы, которые будут переданы в конструктор APIView.

3. DynamicRoute - объект, который описывает динамический маршрут (со своим URL-паттерном).
4. url - шаблон URL для маршрута, в котором можно использовать переменные, обозначенные фигурными скобками.
5. name - имя маршрута, которое можно использовать для генерации URL.
6. detail - определяет, является ли маршрут детальным (с переменной в URL) или нет.
7. initkwargs - дополнительные аргументы, которые будут переданы в конструктор APIView.

In [40]:
from rest_framework.routers import DefaultRouter, Route, DynamicRoute
class TestRouter(DefaultRouter):
    routes = [
        Route(
            url=r'^{prefix}{trailing_slash}$',
            mapping={'get': 'list'},
            name='{basename}-list',
            detail=False,
            initkwargs={'suffix': 'List'}
        ),
        Route(
            url=r'^{prefix}/{lookup}{trailing_slash}$',
            mapping={'get': 'retrieve'},
            name='{basename}-detail',
            detail=True,
            initkwargs={'suffix': 'Detail'}
        ),
        DynamicRoute(
            url=r'^{prefix}/{lookup}/{url_path}{trailing_slash}$',
            name='{basename}-{url_name}',
            detail=True,
            initkwargs={}
        )
    ]